In [1]:
import pandas as pd
import numpy as np
import os
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler, Subset
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import torch.optim as optim
import shutil
import torch.nn.functional as F
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [4]:
# Function to move files to respective directories
def move_files(file_paths, labels, destination):
    for file_path, label in zip(file_paths, labels):
        label_dir = os.path.join(destination, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.move(file_path, label_dir)

# Output directory to store the split data
output_dir = 'HistopathologySplit'

In [4]:
# Create training, validation, and test directories
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

# Initialize lists for image paths and labels
image_paths = []
labels = []

# Collect image file paths and labels
for patient_id in os.listdir(base_dir):
    patient_path = os.path.join(base_dir, patient_id)
    
    if os.path.isdir(patient_path):
        for label in ['0', '1']:
            label_dir = os.path.join(patient_path, label)
            if os.path.isdir(label_dir):
                for image_name in os.listdir(label_dir):
                    image_path = os.path.join(label_dir, image_name)
                    image_paths.append(image_path)
                    labels.append(label)

# Create a stratified train-validation-test split (20% for validation, 10% for test)
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.3, stratify=labels, random_state=42
)

# Further split the temp set into validation and test (2/3 for validation, 1/3 for test)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=1/3, stratify=temp_labels, random_state=42
)

# Move training, validation, and test files
move_files(train_paths, train_labels, train_dir)
move_files(val_paths, val_labels, val_dir)
move_files(test_paths, test_labels, test_dir)

print(f"Training, validation, and test datasets created at '{output_dir}'.")
print(f"Number of training images: {len(train_paths)}")
print(f"Number of validation images: {len(val_paths)}")
print(f"Number of test images: {len(test_paths)}")

Training, validation, and test datasets created at 'HistopathologySplit'.
Number of training images: 194266
Number of validation images: 55505
Number of test images: 27753


In [2]:
# Define image transformations for the training and validation datasets
train_transforms = transforms.Compose([
    transforms.Resize((50, 50)), 
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])                                       
])

val_transforms = transforms.Compose([
    transforms.Resize((50, 50)), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((50, 50)), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
# Load the dataset with the defined transformations
train_dataset = datasets.ImageFolder(root=f'{output_dir}/train', transform=train_transforms)
val_dataset = datasets.ImageFolder(root=f'{output_dir}/val', transform=val_transforms)
test_dataset = datasets.ImageFolder(root=f'{output_dir}/test', transform=test_transforms)

# Extract class labels from the dataset
targets = train_dataset.targets

# Calculate class weights, which are the inverse of class frequencies. Classes with fewer samples will get assigned a higher 
# weight (ensuring that the minority class receives a higher weight, making it more likely to be sampled during training).
class_counts = np.bincount(targets)  # Count the number of samples per class
class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)

sample_weights = torch.tensor([class_weights[label] for label in targets], dtype=torch.float)

# Create the WeightedRandomSampler
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Create DataLoaders with the sampler for training
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=4)

# Create DataLoader for validation and test without a sampler
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Print sample stats (optional)
print(f"Number of training samples: {len(train_loader.dataset)}")
print(f"Number of validation samples: {len(val_loader.dataset)}")
print(f"Number of test samples: {len(test_loader.dataset)}")

/tmp/ipykernel_5641/4242467693.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)


Number of training samples: 194266
Number of validation samples: 55505
Number of test samples: 27753


In [9]:
class CNN(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN, self).__init__()

        # First Convolutional Block for 3 input channels RGB
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm2d(32) #Normalizes the convolution output to have a mean close to 0 and a standard deviation close to 1.
        self.dropout1 = nn.Dropout2d(0.5)  # Apply dropout on feature maps

        # Second Convolutional Block
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.dropout2 = nn.Dropout2d(0.5)  # Apply dropout on feature maps

        # Third Convolutional Block
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm2d(128)
        self.dropout3 = nn.Dropout2d(0.5)  # Apply dropout on feature maps

        # Fully Connected Layer Block. Second FC layer will output the number of classes.
        self.fc1 = nn.Linear(128 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout_fc = nn.Dropout(0.5)  # Apply dropout after fully connected layer

        # Leaky ReLU Activation
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.01)

    def forward(self, x):
        # Apply the first convolutional block
        x = self.leaky_relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = self.dropout1(x)

        # Apply the second convolutional block
        x = self.leaky_relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = self.dropout2(x)

        # Apply the third convolutional block
        x = self.leaky_relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = self.dropout3(x)

        # Flatten the output of the convolutional layers
        x = x.view(-1, 128 * 6 * 6)

        # Fully connected layers
        x = self.leaky_relu(self.fc1(x))
        x = self.dropout_fc(x)
        x = self.fc2(x)

        return x


In [10]:
# Instantiate model, define loss function and optimizer
best_model_path = "best_model.pth"
model = CNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001) # Add L2 regularization to the optimizer
num_epochs = 5

In [12]:
# Training loop
best_val_loss = float('inf')
print("Beginning training!")
for epoch in range(num_epochs):
    print("Starting epoch:", epoch+1)
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    train_accuracy = 100 * correct_train / total_train
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

    # Validation step
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    val_accuracy = 100 * correct_val / total_val
    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%\n")

    # Save the model if validation loss has improved
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"Model saved with validation loss: {avg_val_loss:.4f}")

Beginning training!
Starting epoch: 1
Epoch [1/5], Loss: 0.3886, Accuracy: 77.95%
Validation Loss: 0.5952, Accuracy: 74.75%

Model saved with validation loss: 0.5952
Starting epoch: 2
Epoch [2/5], Loss: 0.3869, Accuracy: 78.12%
Validation Loss: 0.5161, Accuracy: 76.32%

Model saved with validation loss: 0.5161
Starting epoch: 3
Epoch [3/5], Loss: 0.3859, Accuracy: 78.06%
Validation Loss: 0.5136, Accuracy: 76.78%

Model saved with validation loss: 0.5136
Starting epoch: 4
Epoch [4/5], Loss: 0.3848, Accuracy: 78.07%
Validation Loss: 0.5755, Accuracy: 73.32%

Starting epoch: 5
Epoch [5/5], Loss: 0.3847, Accuracy: 78.20%
Validation Loss: 0.5040, Accuracy: 78.01%

Model saved with validation loss: 0.5040


In [13]:
# Load the best model weights before validation
model.load_state_dict(torch.load(best_model_path, weights_only=True))
print("Loaded best model weights from validation.")

# Validation loop
model.eval()
test_loss = 0.0
correct = 0
total = 0

all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Get predictions and actual labels
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

        # Calculate accuracy
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

Loaded best model weights from validation.


In [14]:
# Adding metrics of interest
avg_test_loss = test_loss / len(test_loader)
accuracy = 100 * correct / total
print(f"Test Loss: {avg_test_loss:.4f} \nAccuracy: {accuracy:.2f}%")

# Calculate precision, F1 score, and confusion matrix
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"Precision: {precision:.4f} \nRecall: {recall:.4f} \nF1 Score: {f1:.4f}")

Test Loss: 0.5055 
Accuracy: 78.34%
Precision: 0.8453 
Recall: 0.7834 
F1 Score: 0.7937
